<a href="https://colab.research.google.com/github/AnastasiyaKrokhina/DataCon-time-1/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymatgen

Что еще есть наподобие pymatgen?

In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
import sklearn
from sklearn import preprocessing
from sklearn import metrics

# 1. Разработка дескрипторов материалов

В имеющихся данных только одно свойство, напрямую связанное с  химической формулой материла, Syngony, причем у различных составов, а также разных модификаций оксида железа, этот параметр одинаковый, и, соответственно, не дает достаточной информации, связанной с химическим составом материала.

Какие признаки можно добавить: **количество различных хим. элеменов (металлов) в составе**, **количество атомов кислорода**, **средняя степень окисления ионов металлов**, **(магнитный момент материала**, **энергия образования, пока не получилось, не разобралась с pymatgen)**.

In [ ]:
df=pd.read_csv('task 3.csv')

In [ ]:
df.sample(5)

,formula,Km,Kcat,Syngony,length,width,depth,pol,surf,Subtype,ph,temp,Cper,Csub,Ccat
171,CuO,41.000,0.0027988,2,6.0,6.0,6.0,0,0,H2O2,4.0,37.0,1500.0,0.67,4.00
150,CeO2,0.620,0.0000337,7,215.0,50.0,50.0,0,0,H2O2+ATP,7.4,32.5,3.5,0.30,160.00
174,CuO,85.600,0.0000595,2,30.0,30.0,30.0,0,0,H2O2,4.0,40.0,1500.0,0.67,40.00
210,LaCoO3,0.240,8.579816,5,70.0,70.0,70.0,0,1,TMB,6.0,25.0,100.0,25.00,0.01
15,MnFe2O4,0.112,3 298.15,7,4.0,4.0,4.0,1,0,H2O2,3.5,37.0,10.0,1.50,0.50


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   formula  227 non-null    object 
 1   Km       227 non-null    float64
 2   Kcat     227 non-null    object 
 3   Syngony  227 non-null    int64  
 4   length   227 non-null    float64
 5   width    227 non-null    float64
 6   depth    227 non-null    float64
 7   pol      227 non-null    int64  
 8   surf     227 non-null    int64  
 9   Subtype  227 non-null    object 
 10  ph       227 non-null    float64
 11  temp     227 non-null    float64
 12  Cper     227 non-null    float64
 13  Csub     227 non-null    float64
 14  Ccat     227 non-null    float64
dtypes: float64(9), int64(3), object(3)
memory usage: 26.7+ KB


In [ ]:
df['Kcat'] = df['Kcat'].str.replace(' ', '')

In [ ]:
df['Kcat']=df['Kcat'].astype('float')

In [ ]:
from pymatgen.ext.matproj import MPRester

In [ ]:
MAPI_KEY = 'yzlGGAhooZ9pPX1Vu25T'

In [ ]:
mpr = MPRester(MAPI_KEY)

Проверяю выдает ли вообще id и какие-то свойства





In [ ]:
mpr.get_materials_ids('MnFe2O4')[0]

'mp-697842'

In [ ]:
new_descriptors = mpr.query(criteria = 'mp-697842', properties  =['total_magnetization', 'formation_energy_per_atom',
                                                          'nelements', 'energy_per_atom',
                                                          'volume', 'oxide_type'])

Интерпретируемость: c помощью значения намагниченности можно различить магнитные оксиды, например, Co3O4 и Fe3O4;
объем ячейки элементарной ячейки также связан с хим составом;
является материал оксидом или нет, разные энергетические характеристики вроде как влияют на катализ. Характеристик, связанных с зарядом не было

In [ ]:
print(new_descriptors)

[{'total_magnetization': 8.60000003, 'formation_energy_per_atom': -1.8783560068021676, 'nelements': 3, 'energy_per_atom': -7.012086331714286, 'volume': 807.1543697035366, 'oxide_type': 'oxide'}]


In [ ]:
mpr.get_materials_ids('NiPd')[0]

'mp-1220057'

In [ ]:
new_descriptors = mpr.query(criteria = 'mp-1220057', properties  =['total_magnetization', 'formation_energy_per_atom',
                                                          'nelements', 'energy_per_atom', 'band_gap',
                                                          'volume', 'oxide_type'])

In [ ]:
print(new_descriptors)

[{'total_magnetization': 0.9622043, 'formation_energy_per_atom': 0.05664135000000048, 'nelements': 2, 'energy_per_atom': -5.42151302, 'band_gap': 0.0, 'volume': 26.83871717588212, 'oxide_type': 'None'}]


Хочу создать новый столбец id, чтобы с его помощью потом получить новые свойства

In [ ]:
df["material_id"]=df['formula'].apply(lambda row: mpr.get_materials_ids(row)[0] 
                                      if (row not in ['y-Fe2O3', 'a-Fe2O3','Au-CeO2',
                                                      'Au-Cu2O', 'α-AgVO3', 'β-AgVO3', 
                                                      'Cu0.89Zn0.11O', '0.10CeO2/CoO', 
                                                      'CoO/CeO2', 'Co3O4/CeO2', 'Fe3O4-CeO2', 
                                                      'Pd/CeO2', 'Ce-Fe3O4', 'Fe3O4–MnO2', 'Pt-WO2']) else row)

Не получается, выдается ошибка 404, если данного нет (хотя все странные случаи вроде выписала)

**Придется создавать вручную: оксид или нет, средняя степень окисления металла (не получается случаи без индексов обработать)**.

In [ ]:
df["if_oxide"]=df['formula'].apply(lambda row: 1 if ('O' in row) else 0)

In [ ]:
df.head()

,formula,Km,Kcat,Syngony,length,width,depth,pol,surf,Subtype,ph,temp,Cper,Csub,Ccat,if_oxide
0,CoFe2O4,0.00645,12306.45,7,4.1,4.1,4.1,1,1,TMB,4.0,37.0,100.0,15.0,0.02624,1
1,CoFe2O4,0.05537,2363.29,7,13.8,13.8,13.8,1,1,TMB,4.0,37.0,100.0,15.0,0.02624,1
2,CoFe2O4,0.03551,74779.37,7,4.1,4.1,4.1,1,1,TMB,4.0,37.0,100.0,15.0,0.02624,1
3,CoFe2O4,0.22769,3918.25,7,13.8,13.8,13.8,1,1,TMB,4.0,37.0,100.0,15.0,0.02624,1
4,CoFe2O4,0.01725,9184.91,7,24.5,24.5,24.5,1,1,H2O2,4.0,37.0,100.0,15.0,0.02624,1


**Обучение модели**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
X=df.drop(['Km', 'Kcat', 'formula'], axis=1)
y=df['Km']

In [ ]:
model = CatBoostRegressor()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.GridSearchCV(grid, X_train, y_train)

TypeError: ignored

In [ ]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_val, y_val,
                            n_repeats=30,
                            random_state=0)

for i in r.importances_mean.argsort()[::-1]:
     if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
         print(f"{diabetes.feature_names[i]:<8}"
               f"{r.importances_mean[i]:.3f}"
               f" +/- {r.importances_std[i]:.3f}")